In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import csv
import os

# 1. Scrape `go2senkyo.com` for candidate information
## 1.1. Assembly election candidates

In [2]:
a_url = "https://go2senkyo.com/local/senkyo/21754"
a_response = requests.get(a_url)
a_soup = BeautifulSoup(a_response.content, "html.parser")

In [3]:
a_candidates = []
for candidate in a_soup.find_all("h2", class_="m_senkyo_result_data_ttl"):
    dict = {}
    dict['name'] = candidate.find("a").text
    dict['katakana'] = candidate.find("a").find("span").text
    a_candidates.append(dict)

In [4]:
candidate_num = 0
for candidate_demo in a_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    print(f"candidate {candidate_num}")
    index = 0
    for info in candidate_demo.find("p", class_="m_senkyo_result_data_para").find_all("span"):
        if index == 0:
            a_candidates[candidate_num]['age_sex'] = info.text
            index += 1
        if index == 1:
            a_candidates[candidate_num]['new_old'] = info.text
    candidate_num += 1

candidate 0
candidate 1
candidate 2
candidate 3
candidate 4
candidate 5
candidate 6
candidate 7
candidate 8
candidate 9
candidate 10
candidate 11
candidate 12
candidate 13
candidate 14
candidate 15
candidate 16
candidate 17
candidate 18
candidate 19
candidate 20
candidate 21
candidate 22
candidate 23
candidate 24
candidate 25
candidate 26
candidate 27
candidate 28
candidate 29
candidate 30
candidate 31
candidate 32
candidate 33
candidate 34
candidate 35
candidate 36
candidate 37
candidate 38
candidate 39
candidate 40
candidate 41
candidate 42
candidate 43
candidate 44
candidate 45
candidate 46
candidate 47
candidate 48
candidate 49
candidate 50
candidate 51


In [5]:
candidate_num = 0
for candidate_party in a_soup.find_all("div", class_="m_senkyo_result_data_bottom_left"):
    a_candidates[candidate_num]['party'] = candidate_party.find("p", class_="m_senkyo_result_data_circle").text
    candidate_num += 1

In [6]:
candidate_num = 0
for candidate_work in a_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    work = candidate_work.find("p", class_="m_senkyo_result_data_para small").text
    if work:
        a_candidates[candidate_num]['vocation'] = work
    else:
        a_candidates[candidate_num]['vocation'] = "NA"
    candidate_num += 1

In [7]:
a_candidates

[{'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t金井 たかしカナイ タカシ\n',
  'katakana': 'カナイ タカシ',
  'age_sex': '60歳  (男) ',
  'new_old': '新人',
  'party': '自由民主党',
  'vocation': '弁護士'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t太田 あつしオオタ アツシ\n',
  'katakana': 'オオタ アツシ',
  'age_sex': '48歳  (男) ',
  'new_old': '新人',
  'party': '無所属',
  'vocation': '会社員'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t岩田 まさかずイワタ マサカズ\n',
  'katakana': 'イワタ マサカズ',
  'age_sex': '49歳  (男) ',
  'new_old': '現職',
  'party': '無所属',
  'vocation': '江戸川区議会・区議会江戸川クラブ（幹事長）'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t間宮 由美マミヤ ユミ\n',
  'katakana': 'マミヤ ユミ',
  'age_sex': '60歳  (女) ',
  'new_old': '現職',
  'party': '無所属',
  'vocation': '江戸川区議会議員'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t丸山 れいこマルヤマ レイコ\n',
  'katakana': 'マルヤマ レイコ',
  'age_sex': '70歳  (女) ',
  'new_old': '新人',
  'party': '日本維新の会',
  'vocation': 'NA'},
 {'name': '\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t五十嵐 まさおイガラシ マサオ\n',
  'katakana': 'イガラシ マサオ',
  'age_sex': '3

In [8]:
a_candidates_df = pd.DataFrame(a_candidates)
a_candidates_df['category'] = 'A'

In [9]:
a_candidates_df

,name,katakana,age_sex,new_old,party,vocation,category
0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t金井 たかしカナイ タカシ\n,カナイ タカシ,60歳 (男),新人,自由民主党,弁護士,A
1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t太田 あつしオオタ アツシ\n,オオタ アツシ,48歳 (男),新人,無所属,会社員,A
2,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t岩田 まさかずイワタ マ...,イワタ マサカズ,49歳 (男),現職,無所属,江戸川区議会・区議会江戸川クラブ（幹事長）,A
3,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t間宮 由美マミヤ ユミ\n,マミヤ ユミ,60歳 (女),現職,無所属,江戸川区議会議員,A
4,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t丸山 れいこマルヤマ レ...,マルヤマ レイコ,70歳 (女),新人,日本維新の会,NA,A
5,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t五十嵐 まさおイガラシ ...,イガラシ マサオ,38歳 (男),新人,参政党,NA,A
6,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t勝山 まゆみカツヤマ マ...,カツヤマ マユミ,54歳 (女),新人,自由民主党,自由民主党東京都江戸川区第十一支部長,A
7,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t肌野 ゆうじハダノ ユウジ\n,ハダノ ユウジ,53歳 (男),新人,自由民主党,NA,A
8,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tしかくら 勇シカクラ イ...,シカクラ イサム,58歳 (男),現職,自由民主党,江戸川区議会自民党幹事,A
9,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t福本 みつひろフクモト ...,フクモト ミツヒロ,51歳 (男),現職,自由民主党,江戸川区議会議長,A


## 1.2 Mayor election candidates

In [10]:
m_url = "https://go2senkyo.com/local/senkyo/21753"
m_response = requests.get(m_url)
m_soup = BeautifulSoup(m_response.content, "html.parser")

In [11]:
m_candidates = []
for candidate in m_soup.find_all("h2", class_="m_senkyo_result_data_ttl"):
    dict = {}
    dict['name'] = candidate.find("a").text
    dict['katakana'] = candidate.find("a").find("span").text
    m_candidates.append(dict)

In [12]:
candidate_num = 0
for candidate_demo in m_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    print(f"candidate {candidate_num}")
    index = 0
    for info in candidate_demo.find("p", class_="m_senkyo_result_data_para").find_all("span"):
        if index == 0:
            m_candidates[candidate_num]['age_sex'] = info.text
            index += 1
        if index == 1:
            m_candidates[candidate_num]['new_old'] = info.text
    candidate_num += 1

candidate 0
candidate 1


In [13]:
candidate_num = 0
for candidate_party in m_soup.find_all("div", class_="m_senkyo_result_data_bottom_left"):
    m_candidates[candidate_num]['party'] = candidate_party.find("p", class_="m_senkyo_result_data_circle").text
    candidate_num += 1

In [14]:
candidate_num = 0
for candidate_work in m_soup.find_all("div", class_="m_senkyo_result_data_bottom_right"):
    work = candidate_work.find("p", class_="m_senkyo_result_data_para small").text
    if work:
        m_candidates[candidate_num]['vocation'] = work
    else:
        m_candidates[candidate_num]['vocation'] = "NA"
    candidate_num += 1

In [15]:
m_candidates_df = pd.DataFrame(m_candidates)
m_candidates_df['category'] = "M"

In [16]:
m_candidates_df

,name,katakana,age_sex,new_old,party,vocation,category
0,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t斉藤 たけしサイトウ タ...,サイトウ タケシ,60歳 (男),現職,無所属,NA,M
1,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tセバタ 勇セバタ イサム\n,セバタ イサム,73歳 (男),新人,無所属,江戸川区議会議員（10期40年）,M


## 1.3 Clean scraped data

In [17]:
# merge assembly and mayor candidates dataframes
candidates_df = pd.concat([m_candidates_df, a_candidates_df], ignore_index=True)

In [18]:
# clean "name" column
candidates_df['name_cleaned'] = candidates_df.apply(
    lambda x: x['name'].strip('\n').strip('\t').replace(x['katakana'], ""), axis=1
)
candidates_df = candidates_df.drop("name", axis=1).rename(columns={'name_cleaned': 'name'})

In [19]:
# create "age" column from "age_sex"
candidates_df['age'] = candidates_df.age_sex.apply(lambda x: re.findall(r".+歳", x)[0].replace('歳', "").replace("-", "NA"))

In [20]:
# create "sex" column from "age_sex"
candidates_df['sex'] = candidates_df.age_sex.apply(lambda x: re.findall(r"\(([^)]*)\)", x)) 
candidates_df['sex'] = [item[0] if len(item) > 0 else "NA" for item in candidates_df.sex]
candidates_df['sex'] = np.where(
    candidates_df.sex == "男", "M", np.where(
        candidates_df.sex == "女", "F", np.where(
            candidates_df.sex == "", "NA", "NA"
            )
        )
    )

In [21]:
# remove "age_sex" column
candidates_df = candidates_df.drop("age_sex", axis=1)

In [22]:
candidates_df

,katakana,new_old,party,vocation,category,name,age,sex
0,サイトウ タケシ,現職,無所属,NA,M,斉藤 たけし,60,M
1,セバタ イサム,新人,無所属,江戸川区議会議員（10期40年）,M,セバタ 勇,73,M
2,カナイ タカシ,新人,自由民主党,弁護士,A,金井 たかし,60,M
3,オオタ アツシ,新人,無所属,会社員,A,太田 あつし,48,M
4,イワタ マサカズ,現職,無所属,江戸川区議会・区議会江戸川クラブ（幹事長）,A,岩田 まさかず,49,M
5,マミヤ ユミ,現職,無所属,江戸川区議会議員,A,間宮 由美,60,F
6,マルヤマ レイコ,新人,日本維新の会,NA,A,丸山 れいこ,70,F
7,イガラシ マサオ,新人,参政党,NA,A,五十嵐 まさお,38,M
8,カツヤマ マユミ,新人,自由民主党,自由民主党東京都江戸川区第十一支部長,A,勝山 まゆみ,54,F
9,ハダノ ユウジ,新人,自由民主党,NA,A,肌野 ゆうじ,53,M


# 2. Search Twitter accounts of the candidates

1. Manually searched a candidate's name on Twitter
2. If not found, searched the name + "江戸川" (Edogawa) on Twitter
3. If not found, searched the name + "江戸川" on Google. Look for Twitter account in the result. If Twitter account is not found but their website/blog etc. is found, looked for Twitter id on it
4. If still not found, `"NA"` 

In [23]:
twitter_ids = [
    "3sedogawa", "jcpedo_sbata",
    "edogawafuture", "maruyamareiko3", "Iwata_Masakazu", "kanai_jsc", "masao_igarashi", 
    "mamiyayumi", "yw8pl21", "Hadano_Yuji", "NA", "officefukumoto", 
    "sjj48_honoka", "ken16000", "Ks9CcUOcZ0NExoa", "NA", "NA", 
    "omiram33", "hayashiakiko_", "kobayashitomoo_", "Henry66239746", "TakizawaYasuko", 
    "sasayan210", "NA", "NA", "nozakimakoto77", "NA", 
    "tanakajuichi", "Shinchan1122JFK", "hayakawa_kazue", "NA", "NA", 
    "edogawasasaki", "kometokoro", "YWWp826zmWNf7g", "K328n", "59Hori", 
    "sekine_komei", "W6B7M4s57S9JG4I", "TakasiNakamichi", "NA", "NA", 
    "kazu3816", "kimotomayu44", "nanapy_1987", "NA", "komei_kawai", 
    "ryoichiezoe", "edoken", "ReiwaTamura", "omtanoriko", "NA", 
    "kenji_makino", "jcp_oaya"
    ]

In [24]:
candidates_df['twitter_id'] = twitter_ids

In [25]:
candidates_df

,katakana,new_old,party,vocation,category,name,age,sex,twitter_id
0,サイトウ タケシ,現職,無所属,NA,M,斉藤 たけし,60,M,3sedogawa
1,セバタ イサム,新人,無所属,江戸川区議会議員（10期40年）,M,セバタ 勇,73,M,jcpedo_sbata
2,カナイ タカシ,新人,自由民主党,弁護士,A,金井 たかし,60,M,edogawafuture
3,オオタ アツシ,新人,無所属,会社員,A,太田 あつし,48,M,maruyamareiko3
4,イワタ マサカズ,現職,無所属,江戸川区議会・区議会江戸川クラブ（幹事長）,A,岩田 まさかず,49,M,Iwata_Masakazu
5,マミヤ ユミ,現職,無所属,江戸川区議会議員,A,間宮 由美,60,F,kanai_jsc
6,マルヤマ レイコ,新人,日本維新の会,NA,A,丸山 れいこ,70,F,masao_igarashi
7,イガラシ マサオ,新人,参政党,NA,A,五十嵐 まさお,38,M,mamiyayumi
8,カツヤマ マユミ,新人,自由民主党,自由民主党東京都江戸川区第十一支部長,A,勝山 まゆみ,54,F,yw8pl21
9,ハダノ ユウジ,新人,自由民主党,NA,A,肌野 ゆうじ,53,M,Hadano_Yuji


In [26]:
# save candidates df
candidates_df.to_pickle("data/candidates_df.p")

In [27]:
import os

In [28]:
# write twitter ids to .txt file
if not os.path.exists('data/ids.txt'):
    mode = 'w+'
else:
    mode = 'w'
with open('data/ids.txt', mode) as txtfile:
    txtfile.write('\n'.join(id for id in candidates_df.twitter_id if id != "NA"))

# 3. Get tweets
- Use <a href="https://github.com/marquisvictor/Optimized-Modified-GetOldTweets3-OMGOT">`Get Old Tweets 3`</a> library to collect data - use `helpers/script.py` to write `helpers/commands.sh` and run `helpers/commands.sh`. This saves one csv file per twitter id (`twitter id.csv`) in `data` directory
- Scope of analysis: After last local election (21 Apr 2019) ~ 12 Apr 2023

# 4. Clean downloaded csv files

## 4.1. Delimit data with commas

In [30]:
def split_rows(file_path):
    '''format csv files and save them in 'data/cleaned' directory'''
    with open(file_path) as file:
        lst = [["tweet_id", "time", "id", "tweet"]]
        for row in file:
            sublist = []
            sublist.append(re.findall(r"\d{19}", row)[0])
            sublist.append(re.findall(r"\d{4}-\d{2}-\d{2}\s{1}\d{2}:\d{2}:\d{2}", row)[0])
            sublist.append(re.findall(r"<([^>]+)>", row)[0])
            tweet = re.findall(r"\d{19}\s\d{4}-\d{2}-\d{2}\s{1}\d{2}:\d{2}:\d{2}\s\+\d{4}\s<[^>]+>\s([\s\S]+)\n", row)
            sublist.append(tweet[0])
            lst.append(sublist)
    file_name = re.findall(r"data\/(.+)", file_path)[0]
    new_csv_path = f"data/cleaned/{file_name}"
    if os.path.exists(new_csv_path):
        mode = 'w'
    else:
        mode = 'w+'
    with open(new_csv_path, mode) as file:
        writer = csv.writer(file)
        writer.writerows(lst)


In [ ]:
ids = [id for id in candidates_df.twitter_id if id != "NA"]

In [ ]:
for id in ids:
    split_rows(f"data/{id}.csv")